# Generacion de Grad-CAM para pytorch en modelo de referencia ResNet50

### Alumno : Gustavo Ayrton Bitocchi
### Director : Diego Alexis Evin
### Universidad Austral Cohorte 2020/21
### Trabajo final de Maestria
------------------------------------------------------------------------------------------------------

## Bibliotecas

In [ ]:
# Instalacion de biblioteca

!pip install grad-cam --quiet

In [ ]:
# Importamos bibliotecas

import cv2
import torch
import numpy as np
import torchvision.transforms as tt

from pytorch_grad_cam import FullGrad
from pytorch_grad_cam.utils.image import show_cam_on_image

## Definicion de funciones

In [ ]:
# Definimos funcion para cargar modelo generado

def load_model(filepath):
    checkpoint = torch.load(filepath)
    model = checkpoint['model']
    model.load_state_dict(checkpoint['state_dict'])
    for parameter in model.parameters():
        parameter.requires_grad = False
    model.eval()
    return model

In [ ]:
# Definimos funcion que convierte imagen a tensor

def convert_img_to_tensor(img):
    preprocessing = tt.Compose([tt.ToTensor()])
    return preprocessing(img.copy()).unsqueeze(0)

In [ ]:
# Definimos funcion que recorta la imagen dada en el centro

def center_crop(img, dim):
	width, height = img.shape[1], img.shape[0]
	crop_width = dim[0] if dim[0]<img.shape[1] else img.shape[1]
	crop_height = dim[1] if dim[1]<img.shape[0] else img.shape[0] 
	mid_x, mid_y = int(width/2), int(height/2)
	cw2, ch2 = int(crop_width/2), int(crop_height/2) 
	return img[mid_y-ch2:mid_y+ch2, mid_x-cw2:mid_x+cw2]

## Carga de modelo y definicion de capa a computar CAM

In [ ]:
# Cargamos modelo y definimos que capa utilizaremos como objetivo

model = load_model('DIR_MODEL_GENERATED')
target_layers  = [model.network.layer4[-1]]

## Instanciacion de modelo FullGrad

In [ ]:
# Instanciamos modelo FullGrad

cam = FullGrad(model = model, target_layers = target_layers, use_cuda = True)
cam.batch_size = 32

## Lectura y pre-procesamiento de imagen de prueba

In [ ]:
# Leemos imagen de prueba

rgb_img = cv2.imread('DIR_TEST_IMAGE', 1)[:, :, ::-1]

# Realizamos pre-procesamiento a la imagen

rgb_img = cv2.resize(rgb_img, (255, 255))
rgb_img = center_crop(rgb_img, (224, 224))
rgb_img = np.float32(rgb_img) / 255

# Convertimos imagen en tensor de entrada

input_tensor = convert_img_to_tensor(rgb_img)

## Ejecucion de modelo cam y guardado de heatmap generado

In [ ]:
# Ejecutamos modelo CAM con aug_smooth y eigen_smooth para reducir el ruido en las CAMs y hacer que se ajusten mejor a los objetos

grayscale_cam = cam(input_tensor = input_tensor, aug_smooth = True, eigen_smooth = True)

In [ ]:
# Visualizamos CAM en imagen

visualization = show_cam_on_image(rgb_img, grayscale_cam[0, :], use_rgb=True)

In [ ]:
# Guardamos el heatmap generado en un directorio

cv2.imwrite('DIR_SAVE_HEATMAP', visualization)